In [1]:
import tensorflow as tf

import numpy as np
import time


In [2]:
from names import check, raw_data, vectorize, seq_model
from names.rules_based import census, from_file, npc_gen

### Read the data

First, look in the text:

In [3]:
vocab = check.default_vocab()

In [4]:
df_census = census.census()
df_census

,name,gender,count,year,source
0,Mary,F,7065,1880,US census
1,Anna,F,2604,1880,US census
2,Emma,F,2003,1880,US census
3,Elizabeth,F,1939,1880,US census
4,Minnie,F,1746,1880,US census
...,...,...,...,...,...
31266,Zykell,M,5,2020,US census
31267,Zylus,M,5,2020,US census
31268,Zymari,M,5,2020,US census
31269,Zyn,M,5,2020,US census


In [5]:
useall = False
if useall:
    raw = [raw_data.row2sample(df_census.iloc[i]) for i in range(df_census.name.count())]
else:
    raw = raw_data.sample_from_dfs([df_census], k=100000)

In [6]:
raw[-20:]

["{'gender': 'F', 'count': 5, 'year': 2014, 'source': 'US census'}\nRhemy",
 "{'gender': 'M', 'count': 81, 'year': 1904, 'source': 'US census'}\nJuan",
 "{'gender': 'F', 'count': 19, 'year': 2018, 'source': 'US census'}\nEmillia",
 "{'gender': 'M', 'count': 7, 'year': 1905, 'source': 'US census'}\nRoyce",
 "{'gender': 'F', 'count': 6, 'year': 2020, 'source': 'US census'}\nDrishti",
 "{'gender': 'F', 'count': 6, 'year': 1941, 'source': 'US census'}\nObie",
 "{'gender': 'F', 'count': 11, 'year': 2013, 'source': 'US census'}\nLakiya",
 "{'gender': 'F', 'count': 5, 'year': 1952, 'source': 'US census'}\nCharmaigne",
 "{'gender': 'M', 'count': 23, 'year': 2012, 'source': 'US census'}\nKashawn",
 "{'gender': 'F', 'count': 12, 'year': 1989, 'source': 'US census'}\nCarisma",
 "{'gender': 'F', 'count': 13, 'year': 1923, 'source': 'US census'}\nEdithe",
 "{'gender': 'F', 'count': 17, 'year': 1994, 'source': 'US census'}\nKeanda",
 "{'gender': 'F', 'count': 9, 'year': 1893, 'source': 'US census'}\

In [7]:
text = raw_data.join_raw(raw)

In [8]:
text[:300]

"{'gender': 'F', 'count': 16, 'year': 1896, 'source': 'US census'}\nCorene                            \n{'gender': 'F', 'count': 5, 'year': 1953, 'source': 'US census'}\nKai                                \n{'gender': 'M', 'count': 36, 'year': 1927, 'source': 'US census'}\nAnselmo                         "

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [9]:
ds = vectorize.DataSet(text, vocab=vocab)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [10]:
# Length of the vocabulary in chars
# vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = [1024]

In [11]:
chk = check.CheckPoint("census_model")
model = chk.load()
if model is not None:
    print("model loaded")
else:
    print("creating new")
    model = seq_model.SeqModel(vocab,
        embedding_dim=embedding_dim,
        rnn_units=rnn_units)

model loaded


For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [12]:
for input_example_batch, target_example_batch in ds.dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 179) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [13]:
model.summary()

Model: "seq_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  45824     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  183475    
Total params: 4,167,603
Trainable params: 4,167,603
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [14]:
model.compile_model()

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [15]:
EPOCHS = 10

In [16]:
history = model.fit_with_chk(ds, checkpoint=chk, epochs=EPOCHS)

# Epoch 1/10
# 1562/1562 [==============================] - 65s 40ms/step - loss: 0.4260
# Epoch 2/10
# 1562/1562 [==============================] - 65s 41ms/step - loss: 0.2659
# Epoch 3/10
# 1562/1562 [==============================] - 66s 42ms/step - loss: 0.2552
# Epoch 4/10
# 1562/1562 [==============================] - 66s 42ms/step - loss: 0.2475
# Epoch 5/10
# 1562/1562 [==============================] - 66s 42ms/step - loss: 0.2435
# Epoch 6/10
# 1562/1562 [==============================] - 66s 42ms/step - loss: 0.2314
# Epoch 7/10
# 1562/1562 [==============================] - 66s 42ms/step - loss: 0.2265
# Epoch 8/10
# 1562/1562 [==============================] - 66s 42ms/step - loss: 0.2225
# Epoch 9/10
# 1562/1562 [==============================] - 66s 42ms/step - loss: 0.2162
# Epoch 10/10
# 1562/1562 [==============================] - 66s 42ms/step - loss: 0.2172

Epoch 1/10
 943/1562 [=================>............] - ETA: 24s - loss: 0.3432

KeyboardInterrupt: 

In [ ]:
one_step_model = seq_model.OneStep(model)

In [ ]:
chk.save(model)

In [ ]:
def generate_names(stem="", n=1):
    names = []
    for _ in range(n):
        states = None
        next_char = [stem + ch for ch in "QWERTYUIOPASDFGHJKLZXCVBNM"]
        next_char = tf.constant(next_char)
        result = [next_char]

        for n in range(40):
          next_char, states = one_step_model.generate_one_step(next_char, states=states)
          result.append(next_char)

        result = tf.strings.join(result)
        names += [" ".join([a.strip() for a in name.decode("utf-8").split() if len(a)]) for name in result.numpy()]
    return names

In [ ]:
generate_names(stem="{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'}\n")

# ["{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'} Quønøa"



In [ ]:
generate_names(stem="{'gender': 'M', 'count': 10, 'year': 1880, 'source': 'US census'}\n")

In [ ]:
generate_names(stem="{'gender': 'F', 'count': 7065, 'year': 1880, 'source': 'US census'}\n")